Analog of SMTpy. Myabe I should just do this in a project from the get go

Maybe why even bother using my own DSL? reuse Epxpr

Well for programmaticaly invoking z3 it might be nice.

Welllll. The z3 smtlib parser is fine.


Could I take this approach all the way to a WP pass?
substitution might be a pain
macrotools might have something useful for that

using MacroTools
ex = :(x + x^2 + x^3)
MacroTools.postwalk(ex -> ex == :x ? :y : ex, ex)
MacroTools.postwalk(ex -> ex == :x ? 1.5 : ex, ex)

Need to be on the lookout for new forms that introduce bound variables. God what a pain

Why can't we overload programs to run them backwards?

Another way of thinking of wp is as pushing @assert statements backwards through the program.

Weakest precondition of differential programs.
Differential operators are kind of like while loops. It is not obvious when they end.
What is the WP of a differential flow? WP of while loop is answer basically.
The evolution can anively be thought of as a while loop 
Why is it not a for loop? For any given dt, we know how many iterations a simulator will take. Perahsp because the dt to be accurate 
is unknown a priori. A while loop is also a for loop in this sense, except we punt on the exact number of iterations required. The number of iterations is discovered at runtime. We could "skolemize" this condition into a for loop with enough insight.

Making a while loop differential construct.
Programs vs state machines. The main difference is making the program counter explicit. Is there a reason to make 

https://github.com/JuliaCompilerPlugins/Mixtape.jl


Hmmmmm. This new understanding of CHC makes using the IR more compelling.


https://arxiv.org/abs/2010.07763 refinement types : a tutorial 
Interesting shit

Wp for less trivial constructs - dilig
https://www.cs.utexas.edu/~isil/cs389L/HoareLogic3-6up.pdf

multiple dispatch. hmm. typeof(x) construct?

arrays can 
DeclareSort("Ref")
IntArray = ArraySort(Int, Int)
mem = Array("mem", Ref, IntArray)

x and y are references.
x[3] is actually dereference of x, then index of that. two operations.

maxandsum is a nice example

sum <= N*max

http://www.ece.vt.edu/chaowang/pubDOC/iccad08-tutorial-shuvendu-VC.pdf


Hmmm. Interesting. Cody says Why3 generates VCs for like valid pointer checks by instead of dumping into the formula valid(p) /\ Q, it seperately propagates up valid(p) and Q. This is again the classic where do I put the control flow or search. When can I replace metalevel and propsotional and. 

https://www.faculty.ece.vt.edu/chaowang/pubDOC/iccad08-tutorial-shuvendu-VC.pdf Havoc slides. interesting. discusses that leino unstructured program technique. Abstracting a block as being ok if it can't do anything bad. Is this losing path sensitivity or not? Do variables secretly connect different pieces?


Why is WP in any sense superior or preferred to SP. The leino book had an answer i think.

in addition to assert and assume shoul one not have a {} annotation (cut?) that let's you modularize the checks.
WP({X},Q) = {X} with side condition X => Q

Maybe a fun / smart approahc is to make a macro to overload if then else, while, and sequencing.

Macro/function to convert julia type definitions to Z3 ones?

and(x::Z3Bool) == 
and(x,y) = x() && y()
ite(cond::Z3Bool,then,else) = If(cond, then(), else())
ite(cond::Bool,then,else) = if cond then() else else() end


while( c, body ) = 

 c needs to be executable multiple time
closure caputres env at time of first call. Hmm.
while (x == 4) ===>   whileo( () -> x == 4,  body )



In [18]:
begin
x = 4
f = () -> x == 3
f()
x = 3
f() # very weird. So it might kind of work
end

true

In [21]:
mywhile(c,b) = while c() 
    b()
end

x = 4
mywhile(() -> x > 0,
        () -> x -= 1 )


LoadError: UndefVarError: x not defined

In [6]:
@code_typed ((x,y) -> x + (() -> y)())(3,4)

CodeInfo(
1 ─ %1 = Base.add_int(x, y)::Int64
└──      return %1
) => Int64

In [2]:
@gensym x y
x


Symbol("##x#259")

In [ ]:
struct Add
    args
end

struct Times
    args
end

struct Eq
end

struct 

end

struct Var
    name
    typ
end

struct ForAll

end

struct Exists

end


In [51]:
symbol_replace = Dict(
    :! => :not,
    :(==) => :(=),
    :(!=) => :distinct # or do i need something fancier?
)
using MacroTools
function build_let(b,k)
    if length(b) == 0
        return expr_to_smtlib(k)
    end
    l = b[1]
    rest = b[2:end]
    @capture( l, x_ = e_)
    x = expr_to_smtlib(x)
    e = expr_to_smtlib(e)
    r =  build_let(rest, k)
    return "(let (($x  $e)) $r)"
end


function expr_to_smtlib(e)
    #nl = "\n"
    #get_fun_symbols
    e = rmlines(e)
    if e isa Symbol
        if haskey(symbol_replace,e)
            return symbol_replace[e]
        end
        return e
    elseif e isa Number
        return e
    elseif e.head == :call
        #f = e.args[1]
        #@assert f isa Symbol
        args = join(expr_to_smtlib.(e.args), " ")
        return "($args)"
    # elseif e.head == :vect # hmmm
    elseif e.head == :if
        args = expr_to_smtlib.(e.args)
        args = join(args, " ")
        return "(ite $args)"
    elseif e.head == :(&&)
        args = join(expr_to_smtlib.(e.args), " ")
        return "(and $args)"
    elseif e.head == :let # julia lets are actually statements I think. Not great. Also they are sequential.
        println(dump(e))
        if e.args[1].head == :block
            return build_let( e.args[1].args , e.args[2])
        elseif e.args[1].head == :(=)
            return build_let( [e.args[1]], e.args[2] )
        end
    elseif e.head == :(||)
        args = join(expr_to_smtlib.(e.args), " ")
        return "(or $args)"
    elseif e.head == :block
        @assert length(e.args) == 1
        return expr_to_smtlib(e.args[1])
    end
    #nl += "\t"
end
    
function declare_smtlib(x)
    x, typ = x
    #args = join(" ", typ[1:end-1] )
    #res = typ[end]
    if typ == Int
        typ = :Int
    end
    "(declare-const $x   $typ )"
end
function declare_smtlib(x::Vector)
    return join(declare_smtlib.(x), "\n")
end

function parse_model()
end


        
macro smt(ex)
    expr_to_smtlib(ex)
end


@smt 1 + 2 + 3 * 4
#@smt true ? 2 : 3 * 4
@smt !true
@smt true || false
@smt true == false
@smt y - x
@smt(let x = 7, x = 8
        x end)

declare_smtlib( [:x => Int, :y => Int] )

Expr
  head: Symbol let
  args: Array{Any}((2,))
    1: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol x
            2: Int64 7
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol x
            2: Int64 8
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 91
          file: Symbol In[51]
        2: Symbol x
nothing


"(declare-const x   Int )\n(declare-const y   Int )"

In [38]:
typeof(true)

Bool

In [11]:
dump(quote
    let a = 1
        a + b
    end
    end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[11]
    2: Expr
      head: Symbol let
      args: Array{Any}((2,))
        1: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol a
            2: Int64 1
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[11]
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol +
                2: Symbol a
                3: Symbol b


In [105]:
macro func_def(ex)

    println(dump(ex))
    @capture(ex, f_(args__) = d_)
    def = expr_to_smtlib(e)
    "(forall ($args...) $def)"
end
@func_def fact(x) = x == 0 ? 1 : x * fact(x-1)


Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol fact
        2: Symbol x
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 8
          file: Symbol In[105]
        2: Expr
          head: Symbol if
          args: Array{Any}((3,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ==
                2: Symbol x
                3: Int64 0
            2: Int64 1
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol *
                2: Symbol x
                3: Expr
nothing


LoadError: LoadError: type Array has no field head
in expression starting at In[105]:8

In [53]:
dump(:(@assume 3 == 4))
dump(:(@assert 3 == 4))
@Meta.lower x = 3

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @assume
    2: LineNumberNode
      line: Int64 1
      file: Symbol In[53]
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ==
        2: Int64 3
        3: Int64 4
Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @assert
    2: LineNumberNode
      line: Int64 2
      file: Symbol In[53]
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ==
        2: Int64 3
        3: Int64 4


:($(Expr(:thunk, CodeInfo(
    @ none within `top-level scope'
1 ─     x = 3
└──     return 3
))))

In [20]:
let a = 3, a = 4, b = a, println("hello world") # oooook. It's just the syntax tree is allowed. 
    b + 3
end

LoadError: syntax: invalid let syntax around In[20]:1

In [42]:
#= dump(:([2,3]  ))
dump(:( x ? 2 : 3  ))
dump(:( if x 
          2
        else 3
        end))
dump(:(! true))

=#

dump(quote
        #yoyo Hmm. I guess I can't capture comments
    end)
# I could capture line nums though.


Expr
  head: Symbol block
  args: Array{Any}((1,))
    1: LineNumberNode
      line: Int64 13
      file: Symbol In[42]


In [ ]:



quote
@pre x >= 0
function fact(n)
    acc = 1
    x = n
    while x >= 0
        @inv acc*fact(x) == fact(n)
        acc *= x
        x -= 1
    end
end
end

https://docs.julialang.org/en/v1/base/base/#Keywords 
keywords is a nice start for possible syntactic forms


interactively talk to command
@assert_smt
@push
@pop
@get-model
@check-sat
@smt get-model
@smt check-sat
@smt push
@smt pop
@smt 


In [63]:
open(`z3 -in`, "r+") do io
    println(io, "(help)")
    for i in 1:10
        println(readline(io))
    end
end

" (apply <tactic> (<keyword> <value>)*)
    apply the given tactic to the current context, and print the resultant set of goals.
    The following options are available:
      max_memory (unsigned int) (default: infty) maximum amount of memory in megabytes.
      print (bool) (default: true) print resultant goals.
      print_benchmark (bool) (default: false) display resultant goals as a SMT2 benchmark.
      print_dependencies (bool) (default: false) print dependencies when displaying the resultant set of goals.
      print_model_converter (bool) (default: false) print model converter.
      print_statistics (bool) (default: false) print statistics.
      timeout (unsigned int) (default: infty) timeout in milliseconds. (default: 4294967295)


In [85]:
procs = open(`z3 -in`, "r+") 

Process(`z3 -in`, ProcessRunning)

In [88]:
close(procs)

In [ ]:
function smt_assert(proc, e)
    e = smt_of_expr(e)
    write(proc.in, "(assert $e)")
end

function get_model(proc)
    write(proc.in, "(get-model)")
    readline(proc.out)
end

function command(proc, str)
    write(proc.in, "($str)")
    readline(proc.out)
end



In [86]:
peek(procs)

LoadError: InterruptException:

In [75]:
println(procs.in, "(help)")

In [83]:
readline(procs.out)

""

In [81]:
close(procs)

In [80]:
procs

Process(`z3 -in`, ProcessExited(0))

In [ ]:
using MacroTools

function wp(s,Q)
    if @capture(s, begin s1_; s2__ end) && length(s2) > 0
        return translate_statement( Expr(:block, s2...), translate_statement(s1, env, loop_level), loop_level)
    elseif @capture(s, x_ = e_)
        e = smtlib_expr(e)
        return `(let (($x $e)) $Q)`
    elseif @capture(s, if e_ s1_ else s2_ end)
        e = smtlib_expr(e)
        p1 = wp(s1, Q)
        p2 = wp(s2, Q)
        return `(ite $e $p1 $p2)`
    elseif @capture(s, while c_ begin @invariant _i; b__ end end)
        
    end
    println("no match", e)
end
end

In [77]:
using MacroTools
function wp(e, s)
    e = rmlines(e)
    #println(dump(e))
    if e.head == :block
        for c in Iterators.reverse(e.args)
            s = wp(c,s)
        end
        return s
    elseif e.head == :(=) 
        # also need to deal with += etc.
        #check if head is ref to deal with a[3]
        # simultaenous assignemnt
        # Or use let expression?
        return MacroTools.postwalk(ex -> ex == e.args[1] ? e.args[2] : ex, s)
    elseif e.head == :if || e.head == :elseif
        cond, then, els  = e.args
        wpt = wp(then, s)
        wpe = wp(els, s)
        return :( $cond ? $wpt : $wpe )
    elseif e.head == :while
        cond, body = e.args
        @capture(body,  begin @inv inv_  
                               e__ 
                        end )
        #inv = body.args[1]
        s = :( ! $cond && $inv && $s )
    elseif e.head == :call
        args, pre, post = lookuppre_post(e.args[1])
        wp( :(
                
                ) )
    elseif e.head == :macrocall
        # @assert
        # @assume
        # @ghost
        # @havoc
    end
end

wp(quote
    begin
        x = 3
        y = x
        x = 4
    end
        end, :(y == 3))

expr_to_smtlib(wp(quote
    begin
        if x > 0
            y = x
        else
            y = -x
        end
    end
        end, :(y >= 0)))

"(ite (> x 0) (>= x 0) (>= (- x) 0))"

In [2]:
ex = quote
        begin 
            f(x,y)
        end
    
    end
dump(ex)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[2]
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 3
          file: Symbol In[2]
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol f
            2: Symbol x
            3: Symbol y


In [100]:
ex = quote
        begin 
            @inv x > 4
            x += 3
            x -= 7
        end
    
    end

@capture(ex, begin @inv f_  
               e__ 
    end )
e

2-element Vector{Any}:
 :(x += 3)
 :(x -= 7)

In [85]:
dump(
quote
        while x > 3 
            @inv x > 4
            x += 3
        end
    end
)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol In[85]
    2: Expr
      head: Symbol while
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol >
            2: Symbol x
            3: Int64 3
        2: Expr
          head: Symbol block
          args: Array{Any}((4,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[85]
            2: Expr
              head: Symbol macrocall
              args: Array{Any}((3,))
                1: Symbol @inv
                2: LineNumberNode
                3: Expr
            3: LineNumberNode
              line: Int64 4
              file: Symbol In[85]
            4: Expr
              head: Symbol +=
              args: Array{Any}((2,))
                1: Symbol x
                2: Int64 3


In [73]:
dump(:(a[3] = 3))
dump(:(a.foo = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Expr
      head: Symbol ref
      args: Array{Any}((2,))
        1: Symbol a
        2: Int64 3
    2: Int64 3
Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Expr
      head: Symbol .
      args: Array{Any}((2,))
        1: Symbol a
        2: QuoteNode
          value: Symbol foo
    2: Int64 3


In [81]:
dump(
quote
        if x > 3
            5
        elseif x < 2
            6
        else 
            7
        end
    end
)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol In[81]
    2: Expr
      head: Symbol if
      args: Array{Any}((3,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol >
            2: Symbol x
            3: Int64 3
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 4
              file: Symbol In[81]
            2: Int64 5
        3: Expr
          head: Symbol elseif
          args: Array{Any}((3,))
            1: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Int64 6
            3: Expr
              head: Symbol block
              args: Array{Any}((2,))
         

In [69]:
@code_typed 2 + 2

CodeInfo(
1 ─ %1 = Base.add_int(x, y)::Int64
└──      return %1
) => Int64

In [2]:
dump(quote
    while x > 0
        @invariant x == 7
        x = x + 1
    end
    end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[2]
    2: Expr
      head: Symbol while
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol >
            2: Symbol x
            3: Int64 0
        2: Expr
          head: Symbol block
          args: Array{Any}((4,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[2]
            2: Expr
              head: Symbol macrocall
              args: Array{Any}((3,))
                1: Symbol @invariant
                2: LineNumberNode
                3: Expr
            3: LineNumberNode
              line: Int64 4
              file: Symbol In[2]
            4: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol x
                2: Expr


In [3]:
dump(quote
    x[3] = x[4]
    y = x #aliased
    end)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[3]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Expr
          head: Symbol ref
          args: Array{Any}((2,))
            1: Symbol x
            2: Int64 3
        2: Expr
          head: Symbol ref
          args: Array{Any}((2,))
            1: Symbol x
            2: Int64 4
